# Processing Imprints
### Face Dectection for an effect of an fixed focus across imprints of an selph

## Solution


    Using face detection on idle imprint to estimate coordinates that are centered, these are coordinates use to establish a focus window for all imprints of that selph.




In [ ]:
import cv2
import time
import mediapipe as mp
import numpy as np
from scipy.ndimage import zoom
import cv2
import matplotlib.pyplot as plt


In [ ]:
import moviepy.editor as movpy
# !pip install moviepy --upgrade
import os

In [ ]:
# Media pipe face detection class
mp_face_detection = mp.solutions.face_detection
# drawing tools
mp_drawing = mp.solutions.drawing_utils
                                                                                                                                                                                            

In [ ]:
# function to reverse idle and append it the end of the sequence


def reverseIdle(inputVidfile, outfile):
        # videoCapture method of cv2 return video object

    # Pass absolute address of video file
    cap = cv2.VideoCapture(inputVidfile)
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fps = cap.get(cv2.CAP_PROP_FPS)
    # read method of video object will return
    # a tuple with 1st element denotes whether
    # the frame was read successfully or not,
    # 2nd element is the actual frame.
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    writer = cv2.VideoWriter(outfile, fourcc, fps, size)
    # Grab the current frame.
    check , vid = cap.read()

    # counter variable for
    # counting frames
    counter = 0

    # Initialize the value
    # of check variable
    check = True

    frame_list = []

    # If reached the end of the video
    # then we got False value of check.

    # keep looping until we
    # got False value of check.
    while(check == True):

        check , vid = cap.read()

        # Add each frame in the list by
        # using append method of the List
        frame_list.append(vid)

        # increment the counter by 1
        counter += 1

    # last value in the frame_list is None
    # because when video reaches to the end
    # then false value store in check variable
    # and None value is store in vide variable.

    # removing the last value from the
    # frame_list by using pop method of List
    frame_list.pop()

    frame_list.reverse()

    for frame in frame_list:
        writer.write(frame)
    cap.release()
    writer.release()    


### select first and last frame  ✌
### select crop size  ✌
### Draw a  crop rec box center on the face ✌ ✔
### crop that region ✌ ✔
### write it to file✔
### get audio from original file ✔
### return build video file ✔

Clip first and last 2 seconds of video
Collect coordinates of facial bounding box across 4 seconds of video
Find median coordinates of collected coordinates, and average size (W,H) of facial bounding box
Use median coordinates and size of facial bounding box and place in center of cropped frame
Export


In [ ]:
file = 'idle.mp4'

inputVidfile = './selph_input_vids/'+ file
reverseIdle(inputVidfile, "outfile.mp4")


In [ ]:
def detectAndCrop(inputPath):
    cap = cv2.VideoCapture(inputPath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (400, 600)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        while cap.isOpened():

            success, image = cap.read()
            # If loading a video, use 'break' instead of 'continue'.
            if not success:
                cap.release()
#                 writer.release()    
#                 cv2.destroyAllWindows()
                print("detect and  {}".format(inputPath))
                break
            h, w, _ =  image.shape
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = face_detection.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.detections:
                for detection in results.detections:
                    box = detection.location_data.relative_bounding_box
                    rect_start_point = mp_drawing._normalized_to_pixel_coordinates(box.xmin, box.ymin ,w, h)
                    rect_end_point = mp_drawing._normalized_to_pixel_coordinates(box.xmin + box.width, box.ymin + + box.height,w, h)
                    nosep = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP)
                    nosepx = mp_drawing._normalized_to_pixel_coordinates(nosep.x, nosep.y ,w, h)
                    box_mid = int(rect_start_point[0] + 120)
                    m= int(size[0]/2)
                    mp = int(nosepx[0] - m)
                    crop_start = (mp, 100)
                    crop_end = (size[0]+ int(nosepx[0] - m), size[1])
#                     crop = image[crop_start[0]: crop_end[1] + crop_start[0]  , crop_start[0] : crop_end[0] ]
#                     writer.write(crop)    
        cap.release()
    return crop_start , crop_end
        






def justCrop(inputPath, outputPath, crop_start, crop_end):
    cap2 = cv2.VideoCapture(inputPath)
    fps = cap2.get(cv2.CAP_PROP_FPS)
    size = (400, 600)

    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    writer = cv2.VideoWriter(outputPath, fourcc, fps, size)
    while cap2.isOpened():
        
        status, frame = cap2.read()
    
        if not status:
            cap2.release()
            writer.release() 
            print("new crop size", crop.shape, crop_start)
            print('cropping done')
            break
            
        crop = frame[crop_start[0]: crop_end[1] + crop_start[0] , crop_start[0] : crop_end[0] ]
        
        writer.write(crop)
    cap2.release()
    writer.release() 
        
#         crop = clipped_zoom(crop, 1.0)

        

### Folders
#### Input Imprints
- ./selph_input_vids
### for output imprints
- ./

### Processing idle imprint Test

In [ ]:
## do detection and crop 
file_idle = 'idle.mp4'

idleSeq = './selph_input_vids/'+ file_idle
idle_end = 'end{}'.format(file_idle)
idleOut = 'idleout.mp4'

final_idle = 'final_idle.mp4'
print("start detecting start frames")

# detection and returning croping coordinates for idle 
crop_start , crop_end = detectAndCrop(idleSeq)

# cropping idle
justCrop(idleSeq, idleOut,crop_start , crop_end)

reverseIdle(idleOut, idle_end)


# compiling
clip1 = movpy.VideoFileClip(idleOut)
clip2 = movpy.VideoFileClip(idle_end)
# clip3 = movpy.VideoFileClip(endOut)
final_clip = movpy.concatenate_videoclips([clip1,clip2])
final_clip.write_videofile(final_idle)

# removing temp files
os.remove(idle_end)
os.remove(idleOut)
print("OK DONE")


### Imprint  with idle imprint coordinates

In [ ]:

file_idle = 'idle.mp4'

idleSeq = './selph_input_vids/'+ file_idle


file = 'Howareyou.mp4'

inputVidfile = './selph_input_vids/'+ file


rebuildVidFile = 'rebuildv3{}'.format(file)
audiofile = 'audiofile.mp3'
startOut = 'startout.mp4'


print("setting up clips")

clip = movpy.VideoFileClip(inputVidfile)


clip.audio.write_audiofile(audiofile)

print("start detecting start frames")
    
crop_start , crop_end = detectAndCrop(idleSeq)

justCrop(inputVidfile, startOut,crop_start , crop_end)



clip1 = movpy.VideoFileClip(startOut)


audioclip = movpy.AudioFileClip(audiofile)

videoclip = clip1.set_audio(audioclip)
videoclip.write_videofile(rebuildVidFile)



os.remove(audiofile)
os.remove(startOut)

print("OK DONE")


### Following cell are exploring solutions

In [ ]:
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
inputVidfile = './selph_input_vids/selph16.mp4'
audiofile = "selph1322.mp3"
outfile = 'trueselphnoaudio.mp4'
rebuildVidFile = 'rebuildselphselph16v2.mp4'

cap = cv2.VideoCapture(inputVidfile)

print('getting audio..')



my_clip = movpy.VideoFileClip(inputVidfile)


my_clip.audio.write_audiofile(audiofile)
print('getting audio done..')



crop_start = None
crop_end = None

cropping = True

crop_widow_width =  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) - 20 
crop_widow_height =  int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - 40

fps = cap.get(cv2.CAP_PROP_FPS)

print("fps", fps)
                  
# size = ( crop_widow_width, crop_widow_height )
size = (400, 600)

print("size",size)
capture_duration = 5



fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter(outfile, fourcc, fps, size )

# while detecting:
start_time = time.time()

    
while int(time.time() - start_time) < capture_duration :
    success, image = cap.read()
    

#     image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    image.flags.writeable = False

    # Draw the face detection annotations on the image.
    results = face_detection.process(image)
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    
    h, w, _ =  image.shape
    if results.detections:
        for detection in results.detections:
            box = detection.location_data.relative_bounding_box
            rect_start_point = mp_drawing._normalized_to_pixel_coordinates(box.xmin, box.ymin ,w, h)
            rect_end_point = mp_drawing._normalized_to_pixel_coordinates(box.xmin + box.width, box.ymin + + box.height,w, h)
            box_mid = int(rect_start_point[0] + 120)
            
            
            nosep = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP)
            nosepx = mp_drawing._normalized_to_pixel_coordinates(nosep.x, nosep.y ,w, h)
            
            m= int(size[0]/2)
            mp = int(nosepx[0] - m)
            crop_start = (mp, nosepx[1] -  250)


            crop_end = (size[0]+ int(nosepx[0] - m), size[1])



#             m= int(size[0]/2)
#             mp = int(nosepx[0] - m)
#             crop_start = (mp, nosepx[1] -  250)

#             cv2.circle(image, crop_start , 2, (255,0,0), 10)

#             crop_end = (size[0]+ int(nosepx[0] - m), size[1])

#             crop = image[crop_start[0]: crop_end[1] + crop_start[0] , crop_start[0] : crop_end[0] + crop_start[0]]
    
# detecting = False
cap.release()
print('detecting done')
  
print('START cropping ')
cap2 = cv2.VideoCapture(inputVidfile)


while cropping:
    status, frame = cap2.read()
    
    if not status:
        cap2.release()
        writer.release() 
        cropping = False
        print("new crop size", crop.shape, crop_start)
        print('cropping done')
        break

#     crop = frame[crop_start[1]: crop_end[0], crop_start[0]:  crop_end[1]]
#     crop = frame[crop_start[0]: crop_end[1] +crop_start[0], crop_start[1] : crop_end[0]  + crop_start[1]]
#     crop = frame[crop_start[0]: crop_end[1] + crop_start[0] , crop_start[0] : crop_end[0] + crop_start[0]]
    crop = frame[crop_start[0]: crop_end[1] + crop_start[0] , crop_start[0] : crop_end[0] ]
    crop = clipped_zoom(crop, 1.0)
    
    writer.write(crop)

print("rebuilding....")
clip = movpy.VideoFileClip(outfile, audio=False)
  
# loading audio file
audioclip = movpy.AudioFileClip(audiofile)


videoclip = clip.set_audio(audioclip)
videoclip.write_videofile(rebuildVidFile)
print('removing temp files')
os.remove(audiofile)
os.remove(outfile)

print("DONE....")

In [ ]:
cap = cv2.VideoCapture('./selph_input_vids/Goodbye.mp4')

crop_widow_width =  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) - 100
crop_widow_height =  int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) - 50

fps = cap.get(cv2.CAP_PROP_FPS)


                  
size = (400, 600)
# crop_size = (656, 520)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter('newselph1333.mp4', fourcc, fps, size)
# print(size)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.5) as face_detection:
    
    while cap.isOpened():
        
        success, image = cap.read()
        # If loading a video, use 'break' instead of 'continue'.
        if not success:
            cap.release()
            writer.release()    
            cv2.destroyAllWindows()
            print("empty camera frame.")
            break
        h, w, _ =  image.shape
#         print(h,w)                 

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:
#                 mp_drawing.draw_detection(image, detection)
                

                box = detection.location_data.relative_bounding_box
                rect_start_point = mp_drawing._normalized_to_pixel_coordinates(box.xmin, box.ymin ,w, h)
                rect_end_point = mp_drawing._normalized_to_pixel_coordinates(box.xmin + box.width, box.ymin + + box.height,w, h)
         
    
                nosep = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP)
                nosepx = mp_drawing._normalized_to_pixel_coordinates(nosep.x, nosep.y ,w, h)

                box_mid = int(rect_start_point[0] + 120)
                x_start =  int(rect_start_point[0] - 100 )


                y_start =  int(rect_start_point[1] - 200)
                
#                 cv2.circle(image, (box_mid, int(rect_start_point[1] +  (rect_start_point[1] / 2)) ) , 2, (255,0,0), 10)
#                 cv2.line(image, (20,box_mid), (box_mid,box_mid),(255,0,0), 10)
#                 cv2.line(image, (nosepx[0] - (nosepx[0] - 20) ,nosepx[1]), nosepx ,(255,0,150), 10)
#                 cv2.line(image, nosepx, (crop_end[0] - 50, nosepx[1]),(255,200,0), 10)
#                 cv2.line(image, nosepx, (nosepx[0]*2, nosepx[1]),(255,200,0), 10)
    
#                 crop_start = (int(rect_end_point[0]/2) - int(rect_end_point[0]/2) , rect_start_point[1] -  150)
#                 crop_start =( rect_start_point[0] - (rect_start_point[0] - 10), rect_start_point[1] - 150)
                m= int(size[0]/2)
                mp = int(nosepx[0] - m)
        
                                
                crop_start = (mp, 100)

                cv2.circle(image, crop_start , 2, (255,0,0), 10)

                crop_end = (size[0]+ int(nosepx[0] - m), size[1])
#                 crop_end = (size[0], size[1])
#                 cv2.circle(image, crop_end , 2, (255,0,0), 10)

#                 cv2.rectangle(image, rect_start_point , rect_end_point , (255,255,255), 3)

#                 cv2.rectangle(image, crop_start , crop_end , (255,255,255), 3)

                


#                 crop_start = (rect_start_point[0] -110, rect_start_point[1] -200)
                crop = image[crop_start[0]: crop_end[1] + crop_start[0]  , crop_start[0] : crop_end[0] ]
#                 crop = clipped_zoom(crop, 1.0)
                writer.write(crop)    
                print(mp,nosepx, rect_end_point, crop_start , crop_end, crop.shape)
                cv2.rectangle(image, crop_start , crop_end , (255,255,255), 3)
        
                cv2.putText(
                             image, #numpy array on which text is written
                             "{}".format(str(nosepx)), #text
                             (100,50), #position at which writing has to start
                             cv2.FONT_HERSHEY_SIMPLEX, #font family
                             1, #font size
                             (209, 80, 0), #font color
                             3)

#                 cv2.imshow('crop 22', image)
                cv2.imshow('crop', image)
        if cv2.waitKey(5) & 0xFF == 27:
          break
    cap.release()
    writer.release()    
    cv2.destroyAllWindows()
    # print "Video stop"





In [ ]:


file = 'Howareyou.mp4'

inputVidfile = './selph_input_vids/'+ file

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
writer = cv2.VideoWriter('newselph1333.mp4', fourcc, fps, size)

# videoCapture method of cv2 return video object

# Pass absolute address of video file
cap = cv2.VideoCapture(inputVidfile)

# read method of video object will return
# a tuple with 1st element denotes whether
# the frame was read successfully or not,
# 2nd element is the actual frame.

# Grab the current frame.
check , vid = cap.read()

# counter variable for
# counting frames
counter = 0

# Initialize the value
# of check variable
check = True

frame_list = []

# If reached the end of the video
# then we got False value of check.

# keep looping until we
# got False value of check.
while(check == True):
	
	# imwrite method of cv2 saves the
	# image to the specified format.
	cv2.imwrite("frame%d.jpg" %counter , vid)
	check , vid = cap.read()
	
	# Add each frame in the list by
	# using append method of the List
	frame_list.append(vid)
	
	# increment the counter by 1
	counter += 1

# last value in the frame_list is None
# because when video reaches to the end
# then false value store in check variable
# and None value is store in vide variable.

# removing the last value from the
# frame_list by using pop method of List
frame_list.pop()

# looping in the List of frames.
for frame in frame_list:
	
	# show the frame.
	cv2.imshow("Frame" , frame)
	
	# waitkey method to stoping the frame
	# for some time. q key is presses,
	# stop the loop
	if cv2.waitKey(25) and 0xFF == ord("q"):
		break
	
# release method of video
# object clean the input video
cap.release()

# close any open windows
cv2.destroyAllWindows()

# reverse the order of the element
# present in the list by using
# reverse method of the List.
frame_list.reverse()

for frame in frame_list:
	cv2.imshow("Frame" , frame)
	if cv2.waitKey(25) and 0xFF == ord("q"):
		break

cap.release()
cv2.destroyAllWindows()
